# CIA World Factbook
In this project, we will use the CIA World Factbook, a summary of statistics of all countries on Earth. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT MIN(population) AS "Minimum population", 
       MAX(population) AS "Maximum population", 
       MIN(population_growth) AS "Minimum population growth",
       MAX(population_growth) AS "Maximum population growth"
  FROM facts;

Done.


Minimum population,Maximum population,Minimum population growth,Maximum population growth
0,7256490011,0.0,4.02


In [4]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts
                    );

Done.


name
Antarctica


In [5]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                    );

Done.


name
World


In [6]:
%%sql
SELECT MIN(population) AS "Minimum population", 
       MAX(population) AS "Maximum population", 
       MIN(population_growth) AS "Minimum population growth",
       MAX(population_growth) AS "Maximum population growth"
  FROM facts
 WHERE name != "World";

Done.


Minimum population,Maximum population,Minimum population growth,Maximum population growth
0,1367485388,0.0,4.02


## Countries with population above average and area below average

In [7]:
%%sql
SELECT name, population, area 
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != "World")
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name != "World"
              );

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Which Country has the most people?

In [8]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                      WHERE name <> "World"
                    );

Done.


name
China


## Which country has the highest growth rate?

In [9]:
%%sql
SELECT name
  FROM facts
 WHERE population_growth = (SELECT MAX(population_growth)
                              FROM facts
                           );

Done.


name
South Sudan


## Which countries have the highest ratios of water to land?

In [10]:
%%sql
SELECT name, 
       area_water, 
       area_land, 
       CAST(area_water AS FLOAT) / area_land AS ratio
  FROM facts
 ORDER BY ratio DESC
 LIMIT 5;
 

Done.


name,area_water,area_land,ratio
British Indian Ocean Territory,54340,60,905.6666666666666
Virgin Islands,1564,346,4.520231213872832
Puerto Rico,4921,8870,0.5547914317925592
"Bahamas, The",3870,10010,0.3866133866133866
Guinea-Bissau,8005,28120,0.2846728307254623


## Which countries have more water than land?

In [11]:
%%sql
SELECT name, 
       area_water, 
       area_land
  FROM facts
 WHERE area_water > area_land;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


## Which countries will add the most people to their populations next year?

In [12]:
%%sql
SELECT name, population, population_growth, 
      (population * population_growth)/100 AS next_year
  FROM facts
 WHERE name <> "World"
 ORDER BY next_year DESC
 LIMIT 10;

Done.


name,population,population_growth,next_year
India,1251695584,1.22,15270686.1248
China,1367485388,0.45,6153684.246
Nigeria,181562056,2.45,4448270.372
Pakistan,199085847,1.46,2906653.3662
Ethiopia,99465819,2.89,2874562.1691
Bangladesh,168957745,1.6,2703323.92
United States,321368864,0.78,2506677.1392
Indonesia,255993674,0.92,2355141.8008000003
"Congo, Democratic Republic of the",79375136,2.45,1944690.8320000002
Philippines,100998376,1.61,1626073.8536


## Which countries have a higher death rate than birth rate?

In [13]:
%%sql
SELECT name, 
       birth_rate, 
       death_rate
  FROM facts
 WHERE birth_rate < death_rate;

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


## Top 10 most densely populated countries

In [14]:
%%sql
SELECT name, population, area, population / area AS density
  FROM facts
 ORDER BY density DESC
 LIMIT 10;

Done.


name,population,area,density
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299
